## Introduction
This notebook builds predictive models to forecast train delays based on the cleaned SNCF dataset. We'll compare several algorithms and select the best performing model.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# ML libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Set visualization style
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("viridis")

## Loading and Preparing Data
First, let's load the cleaned dataset that we prepared in the EDA notebook.

In [ ]:
# Load the cleaned dataset
print("Loading cleaned dataset...")
df = pd.read_csv("cleaned_dataset.csv")

print(f"Dataset shape: {df.shape}")
print(df.head())

## Feature Selection and Target Definition
Let's define our features and target variable for the predictive model.

In [ ]:
# Define features and target
print("\nPreparing features and target variables...")

# Target variable: Average delay of all trains at arrival
target = "Average delay of all trains at arrival"

# Features to use for prediction
numerical_features = [
    "Average journey time",
    "Number of scheduled trains",
    "Number of cancelled trains",
    "Number of trains delayed at departure",
    "Average delay of late trains at departure",
    "Average delay of all trains at departure",
    "Pct delay due to external causes",
    "Pct delay due to infrastructure",
    "Pct delay due to traffic management",
    "Pct delay due to rolling stock",
    "Pct delay due to station management and equipment reuse",
    "Pct delay due to passenger handling (crowding, disabled persons, connections)",
    "Year",
    "Month",
]

categorical_features = ["Service", "Departure station", "Arrival station", "Season"]

# Filter features that actually exist in the dataframe
numerical_features = [f for f in numerical_features if f in df.columns]
categorical_features = [f for f in categorical_features if f in df.columns]

## Handling Missing Values
Check for and handle any remaining missing values in our selected features.

In [ ]:
# Check for any missing values in our features and target
print("\nChecking missing values in selected features:")
features_df = df[
    numerical_features + categorical_features + [target]
].copy()  # Create a copy
print(features_df.isna().sum())

# Fill missing values
# For numerical features
for col in numerical_features:
    features_df.loc[:, col] = features_df[col].fillna(features_df[col].median())

# For categorical features
for col in categorical_features:
    features_df.loc[:, col] = features_df[col].fillna(features_df[col].mode()[0])

## Train-Test Split
Let's split our data into training and testing sets.

In [ ]:
# Split the data into features and target
X = features_df.drop(target, axis=1)
y = features_df[target]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42
)

print(f"\nTraining set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

## Data Preprocessing
Set up preprocessing pipelines for numerical and categorical features.

In [ ]:
# Create preprocessing for numeric and categorical data
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

## Model Definition and Evaluation Function
Let's define the models we'll compare and create a function to evaluate them.

In [ ]:
# Define and compare multiple models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
}


# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, preprocessor):
    # Create pipeline with preprocessing and model
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

    # Train the model
    pipeline.fit(X_train, y_train)

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    return {
        "model": pipeline,
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "predictions": y_pred,
    }

## Model Training and Comparison
Now let's train all our models and compare their performance.

In [ ]:
# Evaluate all models
print("\nTraining and evaluating models:")
results = {}
for name, model in models.items():
    print(f"Training {name}...")
    results[name] = evaluate_model(
        model, X_train, X_test, y_train, y_test, preprocessor
    )
    print(
        f"{name} - RMSE: {results[name]['rmse']:.2f}, MAE: {results[name]['mae']:.2f}, R²: {results[name]['r2']:.2f}"
    )

## Visualizing Model Comparison
Let's create a visual comparison of our models' performance.

In [ ]:
# Compare model performances
plt.figure(figsize=(12, 6))
model_names = list(results.keys())
rmse_values = [results[name]["rmse"] for name in model_names]
r2_values = [results[name]["r2"] for name in model_names]

x = range(len(model_names))
width = 0.35

plt.bar([i - width / 2 for i in x], rmse_values, width, label="RMSE")
plt.bar([i + width / 2 for i in x], r2_values, width, label="R²")
plt.xlabel("Model")
plt.ylabel("Score")
plt.title("Model Comparison")
plt.xticks(x, model_names)
plt.legend()
plt.tight_layout()
plt.savefig("model_comparison.png")
plt.close()

# Select the best model based on RMSE
best_model_name = min(results, key=lambda k: results[k]["rmse"])
best_model = results[best_model_name]["model"]
print(
    f"\nBest model: {best_model_name} with RMSE: {results[best_model_name]['rmse']:.2f}"
)

## Hyperparameter Tuning
Let's fine-tune our best model to improve its performance further.

In [ ]:
# Fine-tune the best model using Grid Search
print("\nFine-tuning the best model...")

if best_model_name == "Random Forest":
    param_grid = {
        "model__n_estimators": [50, 100, 200],
        "model__max_depth": [None, 10, 20, 30],
        "model__min_samples_split": [2, 5, 10],
    }
elif best_model_name == "Gradient Boosting":
    param_grid = {
        "model__n_estimators": [50, 100, 200],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__max_depth": [3, 5, 7],
    }
elif best_model_name == "Decision Tree":
    param_grid = {
        "model__max_depth": [None, 10, 20, 30],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4],
    }
else:  # Linear Regression
    # LinearRegression doesn't have many hyperparameters to tune
    param_grid = {}

if param_grid:
    grid_search = GridSearchCV(
        best_model, param_grid, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1
    )

    grid_search.fit(X_train, y_train)

    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {-grid_search.best_score_:.2f}")

    # Update best model
    best_model = grid_search.best_estimator_

    # Evaluate tuned model
    y_pred = best_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    print(f"Tuned model - RMSE: {rmse:.2f}, R²: {r2:.2f}")
else:
    print("Skipping hyperparameter tuning for Linear Regression.")

## Feature Importance Analysis
For tree-based models, let's analyze feature importance to understand what factors most influence train delays.

In [ ]:
# Feature importance for the best model (if applicable)
if best_model_name in ["Random Forest", "Gradient Boosting", "Decision Tree"]:
    # Get feature names after preprocessing
    feature_names = []

    # Get numeric feature names (these stay the same)
    feature_names.extend(numerical_features)

    # Get one-hot encoded feature names
    if categorical_features:
        # Fit the preprocessor to get the transformed feature names
        preprocessor.fit(X)

        # Get the one-hot encoder
        ohe = preprocessor.named_transformers_["cat"].named_steps["onehot"]

        # Get all one-hot encoded feature names in one go
        categorical_feature_names = ohe.get_feature_names_out(
            categorical_features
        ).tolist()

        feature_names.extend(categorical_feature_names)

    # Extract feature importances
    importances = best_model.named_steps["model"].feature_importances_

    # Sort feature importances in descending order
    indices = np.argsort(importances)[::-1]

    # Plot the feature importances of the top 15 features
    plt.figure(figsize=(12, 8))
    plt.title(f"Top 15 Feature Importances for {best_model_name}")

    # Get top 15 features
    n_features = min(15, len(feature_names))
    top_indices = indices[:n_features]

    # Create a bar chart
    plt.barh(range(n_features), importances[top_indices], align="center")
    plt.yticks(np.arange(n_features), [feature_names[i] for i in top_indices])
    plt.xlabel("Relative Importance")
    plt.tight_layout()
    plt.savefig("feature_importance.png")
    plt.close()

## Model Performance Visualization
Let's visualize how well our model's predictions match the actual values.

In [ ]:
# Plot actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, best_model.predict(X_test), alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", lw=2)
plt.xlabel("Actual Delay")
plt.ylabel("Predicted Delay")
plt.title(f"Actual vs. Predicted Delays ({best_model_name})")
plt.tight_layout()
plt.savefig("actual_vs_predicted.png")
plt.close()

## Save Model for Dashboard
Finally, let's save our model and relevant information for use in the Streamlit dashboard.

In [ ]:
# Save the model
print("\nSaving the best model...")
with open("tardis_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

# Save the feature lists for later use
model_info = {
    "model_name": best_model_name,
    "numerical_features": numerical_features,
    "categorical_features": categorical_features,
    "target": target,
    "metrics": {"rmse": rmse, "r2": r2},
}

with open("model_info.pkl", "wb") as f:
    pickle.dump(model_info, f)

print(f"Best model ({best_model_name}) saved to 'tardis_model.pkl'")
print("Model information saved to 'model_info.pkl'")
print("\nModel training complete!")